In [ ]:
import os
import numpy as np
import scipy.io
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import matplotlib.pyplot as plt
import torch
import pyro
import pyro.contrib.gp as gp
import math
from sklearn.metrics import mean_squared_error
import copy
from sklearn.preprocessing import StandardScaler
from math import sqrt
from tqdm.notebook import tqdm
from pyro.contrib.gp.likelihoods.likelihood import Likelihood
from pyro.nn.module import PyroParam
import pyro.distributions as dist
from pyro.distributions.torch_distribution import TorchDistributionMixin
from torch.distributions.utils import _standard_normal, broadcast_all
from torch.distributions.exp_family import ExponentialFamily
from pyro.contrib.gp.parameterized import Parameterized
from numbers import Number
from torch.distributions import constraints
from pyro.infer.autoguide import AutoMultivariateNormal, AutoDiagonalNormal,AutoLowRankMultivariateNormal
from pyro.infer import SVI, EmpiricalMarginal, TracePosterior, Trace_ELBO, Predictive,TraceMeanField_ELBO,TraceGraph_ELBO,TracePredictive

from GP_models import GPModel, VariationalGP, VariationalMGP, HGPModel, VariationalMHGP,VariationalPoisGP
from GP_likelihoods import PyroCensoredNormal, CensoredHomoscedGaussian, HomoscedGaussian, CensoredHeteroGaussian, PyroCensoredPois, Censored_Poisson, Poisson
from utils import create_1d_data, censor_1d_data, create_2nd_data, log_error_f
from training_utils import standard_gp_train, censored_gp_train, multi_gp_train, hetero_multi_gp_train, _zero_mean_function

plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)  
plt.rc('legend', fontsize=14)
plt.rc('axes', titlesize=12)
plt.rc('axes', labelsize=14)

# Gaussian Process

### Create data

In [ ]:
torch.manual_seed(49)
np.random.seed(10)
N=100
X1_t,Y1_t,y,noise_scale = create_1d_data(N=100)
X2,Y2,y2,noise_scale_y2 = create_2nd_data(100)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(X1_t,Y1_t,'x',label='Observed')
plt.plot(X1_t,y,'--',label='True')
plt.legend()

### Inference

In [ ]:
intensity = [0.2,0.5,0.8]
file = open('Experiments/Synthetic/SGP/Stats_synthetic.txt', "w")
file1 = open('Experiments/Synthetic/CGP/Stats_synthetic.txt', "w")
file2 = open('Experiments/Synthetic/MGP/Stats_synthetic.txt', "w")
file3 = open('Experiments/Synthetic/HMGP/Stats_synthetic.txt', "w")
for intens in intensity:
    X1,Y1,y_sc,Y1_cens_sc,censoring = censor_1d_data(X1_dat=X1_t,Y1_dat=Y1_t,y_dat=y,int_low = intens)
    standard_gp_train(X1,Y1,Y1_cens_sc,y,y_sc,censoring,intens,noise_scale,file)
    censored_gp_train(X1,Y1,Y1_cens_sc,y,y_sc,censoring,intens,noise_scale,file1)
    X1 = X1.reshape(-1,1)
    X2 = X2.reshape(-1,1)
    X1 = X1.type(torch.float64)
    Y1_cens_sc = Y1_cens_sc.type(torch.float64)
    X2 = X2.type(torch.float64)
    Y2 = Y2.type(torch.float64)
    X_augmented = np.vstack((np.hstack((X1, np.ones_like(X1))), np.hstack((X2, np.zeros_like(X2)))))
    X_augmented = np.hstack((X_augmented,np.vstack((np.zeros_like(X1),np.ones_like(X2)))))
    X_augmented = torch.Tensor(X_augmented)
    Y_augmented = torch.Tensor((np.vstack((Y1_cens_sc,Y2)))).reshape(-1)
    censoring_mul = torch.cat([torch.tensor(censoring),torch.zeros(N).type(torch.int32)])
    multi_gp_train(X_augmented,Y_augmented,X1,Y1,X2,Y2,Y1_cens_sc,y,y2,y_sc,censoring,censoring_mul,intens,noise_scale,file2)
    hetero_multi_gp_train(X_augmented,Y_augmented,X1,Y1,X2,Y2,Y1_cens_sc,y,y2,y_sc,censoring,censoring_mul,intens,noise_scale,file3)

    
file.close()
file1.close()
file2.close()

# Poisson GP

In [ ]:
intensities = [0.2,0.5,0.8]

for intens in intensities:

    N1 = 50
    X1 = np.linspace(0, 20, N1)
    y = np.round(10*np.sin(0.5*X1)+11)


    # Generate noisy observations
    noise_scale = torch.linspace(1,1,N1)*1
    epsilon = torch.distributions.Normal(loc=0, scale=noise_scale).sample()
    Y1 = np.round(y + epsilon.numpy())
    Y1_cens = copy.deepcopy(Y1)

    censoring = np.int32(10*np.sin(0.5*X1) + 11 >= 17) 
    p_c = np.random.uniform(low=intens, high=intens+0.1, size=np.sum(censoring==1))
    Y1_cens[censoring == 1] = np.round(Y1_cens[censoring == 1]*(1-p_c))
    Y1_cens_sc = Y1_cens
    y_sc = y

    X1 = torch.as_tensor(X1).reshape(-1)
    Y1 = torch.as_tensor(Y1).reshape(-1)
    y_sc = torch.as_tensor(y_sc).reshape(-1)
    Y1_cens_sc = torch.as_tensor(Y1_cens_sc).reshape(-1)

    Y1_cens_sc = Y1_cens_sc.type(torch.float64)
    X1 = X1.type(torch.float64)
    censoring = torch.tensor(censoring).type(torch.float64)
    y_sc = y_sc.type(torch.float64)
    
    print('Running standard Poisson with intensity: {}'.format(intens))
    pyro.clear_param_store()
    kern = gp.kernels.RBF(input_dim=1,active_dims=[0],lengthscale=torch.tensor(1.),variance=torch.tensor(1.))
    kern1 = gp.kernels.Linear(input_dim=1,active_dims=[0],variance=torch.tensor(1.))
    full_k = gp.kernels.Sum(kern,kern1)
    response_f = log_error_f
    like_cens = Poisson(response_function=response_f)
    sgphomo = VariationalPoisGP(X=X1, y=Y1_cens_sc, kernel=kern, likelihood=like_cens, mean_function=None,
                     latent_shape=None, whiten=False,jitter=0.05)
    guide = AutoMultivariateNormal(sgphomo.model)
    optimizer = pyro.optim.ClippedAdam({"lr": 0.01,"lrd": 0.99996})
    svi = SVI(sgphomo.model, guide, optimizer, Trace_ELBO(num_particles=10))
    num_epochs = 6000
    losses = []
    pyro.clear_param_store()
    for epoch in tqdm(range(num_epochs)):
        loss = svi.step(X1, Y1_cens_sc)
        losses.append(loss)
        if epoch==num_epochs-1:
            with torch.no_grad():
                predictive = Predictive(sgphomo.model, guide=guide, num_samples=1000,
                return_sites=("f", "g", "_RETURN"))
                samples = predictive(X1)
                f_samples = samples["f"]
                f_mean = f_samples.mean(dim=0)
                f_mean = sgphomo.likelihood.response_function(f_mean)
                #f_std = sgphomo_cens.likelihood.variance.sqrt().item()
                f_std = f_mean.sqrt().detach().numpy()
                f_025 = np.quantile(a=f_samples.detach().numpy(), q=0.025, axis=0)
                f_975 = np.quantile(a=f_samples.detach().numpy(), q=0.975, axis=0)
                fig = plt.figure(figsize=(14,10))
                #fig.add_subplot(121)
                plt.plot(X1.numpy(), y_sc.numpy(), linestyle="--", color="black",label='True function')
                plt.plot(X1.detach().numpy(), f_mean.detach().numpy(), color="black",label='Estimated function')
                plt.fill_between(X1.detach().numpy(), f_mean.detach().numpy() - 1.96*f_std, f_mean.detach().numpy() + 1.96*f_std, alpha=0.3,label='Mean \u00B1 1.96*std.dev')
                plt.fill_between(X1.detach().numpy(), f_025, f_975, alpha=0.3,label='5%/95% Confidence interval')
                plt.scatter(X1.numpy()[censoring==1].reshape(-1,1), y=Y1_cens_sc.numpy()[censoring==1].reshape(-1,1), marker="x", label="Censored Observations", color='#348ABD')
                plt.scatter(X1.numpy()[censoring==0].reshape(-1,1), y=Y1_cens_sc.numpy()[censoring==0].reshape(-1,1), marker="o", label="Non-Censored Observations", color='#348ABD')
                plt.legend(prop={'size': 14})
                plt.ylabel('y')
                plt.xlabel('x')
                plt.savefig('Experiments/Synthetic/Pois/Pois_Synthetic_{}.png'.format(intens))
                
    fig1 = plt.figure(figsize=(8,6))
    plt.plot(losses,label='Loss')
    plt.legend(prop={'size': 14})
    plt.ylabel('ELBO')
    plt.xlabel('Epochs')
    plt.savefig('Experiments/Synthetic/Pois/Pois_Synthetic_Loss_{}.png'.format(intens)) 
    print("RMSE Homosced: ",sqrt(mean_squared_error(y_sc, f_mean.detach().numpy())))
    print("NLPD homosced: ",-(1/len(Y1)*sgphomo.likelihood.y_dist.log_prob(y_sc).sum().item()))
    
    
    print('Running Censored Poisson with intensity: {}'.format(intens))
    pyro.clear_param_store()
    kern = gp.kernels.RBF(input_dim=1,active_dims=[0],lengthscale=torch.tensor(1.),variance=torch.tensor(1.))
    kern1 = gp.kernels.Linear(input_dim=1,active_dims=[0],variance=torch.tensor(1.))
    full_k = gp.kernels.Sum(kern,kern1)
    response_f = log_error_f
    like_cens = Censored_Poisson(censoring=censoring,response_function=response_f)
    sgphomo_cens = VariationalPoisGP(X=X1, y=Y1_cens_sc, kernel=kern, likelihood=like_cens, mean_function=None,
                     latent_shape=None, whiten=False,jitter=0.05)
    guide = AutoMultivariateNormal(sgphomo_cens.model)
    optimizer = pyro.optim.ClippedAdam({"lr": 0.01,"lrd": 0.99996})
    svi = SVI(sgphomo_cens.model, guide, optimizer, Trace_ELBO(num_particles=10))
    num_epochs = 8000
    losses = []
    pyro.clear_param_store()
    for epoch in tqdm(range(num_epochs)):
        loss = svi.step(X1, Y1_cens_sc)
        losses.append(loss)
        if epoch==num_epochs-1:
            with torch.no_grad():
                predictive = Predictive(sgphomo_cens.model, guide=guide, num_samples=1000,
                return_sites=("f", "g", "_RETURN"))
                samples = predictive(X1)
                f_samples = samples["f"]
                f_mean = f_samples.mean(dim=0)
                f_mean = sgphomo_cens.likelihood.response_function(f_mean)
                f_std = f_mean.sqrt().detach().numpy()
                f_025 = np.quantile(a=f_samples.detach().numpy(), q=0.025, axis=0)
                f_975 = np.quantile(a=f_samples.detach().numpy(), q=0.975, axis=0)
                fig = plt.figure(figsize=(14,10))
                plt.plot(X1.numpy(), y_sc.numpy(), linestyle="--", color="black",label='True function')
                plt.plot(X1.detach().numpy(), f_mean.detach().numpy(), color="black",label='Estimated function')
                plt.fill_between(X1.detach().numpy(), f_mean.detach().numpy() - 1.96*f_std, f_mean.detach().numpy() + 1.96*f_std, alpha=0.3,label='Mean \u00B1 1.96*std.dev')
                plt.fill_between(X1.detach().numpy(), f_025, f_975, alpha=0.3,label='5%/95% Confidence interval')
                plt.scatter(X1.numpy()[censoring==1].reshape(-1,1), y=Y1_cens_sc.numpy()[censoring==1].reshape(-1,1), marker="x", label="Censored Observations", color='#348ABD')
                plt.scatter(X1.numpy()[censoring==0].reshape(-1,1), y=Y1_cens_sc.numpy()[censoring==0].reshape(-1,1), marker="o", label="Non-Censored Observations", color='#348ABD')
                plt.legend(prop={'size': 14})
                plt.ylabel('y')
                plt.xlabel('x')
                plt.savefig('Experiments/Synthetic/CensPois/CensPois_Synthetic_{}.png'.format(intens))
                
    fig1 = plt.figure(figsize=(8,6))
    plt.plot(losses,label='Loss')
    plt.legend(prop={'size': 14})
    plt.ylabel('ELBO')
    plt.xlabel('Epochs')
    plt.savefig('Experiments/Synthetic/CensPois/CensPois_Synthetic_Loss_{}.png'.format(intens))
    
    print("RMSE Homosced: ",sqrt(mean_squared_error(y_sc, f_mean.detach().numpy())))
    print("NLPD homosced: ",-(1/len(Y1)*sgphomo_cens.likelihood.y_dist.log_prob(y_sc).sum().item()))
    